In [1]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import CategoricalColorMapper, ColumnDataSource, HoverTool
from bokeh.palettes import Category10, Colorblind, Viridis, Viridis256
from bokeh.transform import linear_cmap
from bokeh.io import output_notebook, export_png
from bokeh.layouts import column, gridplot
output_notebook()

import numpy as np
import numba
import umap
#import umap.constraints as con
from numpy.random import default_rng
rng = default_rng(seed=1234569)


Loading BokehJS ...

In [2]:
factors = [
    ['large', 'small'],
    ['blue','brown','cyan','gray','green','purple','red','yellow'],
    ['metal','rubber'],
    ['cube','cylinder','sphere'],
]
# The factors correspond to distinct 0.0 / 1.0 along individual dimensions
# len(factors)     # 4
nfactors = [ len(factors[i]) for i in range(len(factors)) ]
ndistinct = np.prod(nfactors)
hid = np.sum(nfactors)
print("nfactors", nfactors, "hid", hid, "ndistinct",ndistinct)
# The dimensionality is alread low

nfactors [2, 8, 2, 3] hid 15 ndistinct 96


In [3]:
facprobs = []
for i in range(len(factors)):
    facprobs.extend( [1.0/nfactors[i] for j in range(nfactors[i])] )
print(facprobs)
def create_pool(n=100, d=hid):
    ret = np.zeros((n,d), dtype=np.float32)
    print("ret.shape",ret.shape,"zeros")
    for i in range(n):
        dim = 0
        for j in range(len(factors)):
            sel = rng.choice(nfactors[j])
            print(i,j,dim,sel)
            ret[i,dim+sel] = 1.0
            dim += nfactors[j]
    return ret
def code_up(code):
    """ Cyclic counting in digits base nfactors[] """
    assert( code.shape[0] == len(nfactors) )
    digits = len(nfactors)
    for i in range(digits):
        #carry = 0
        irev = digits - 1 - i
        bump = code[irev] + 1
        if bump < nfactors[irev]:
            code[irev] = bump
            break;
        code[irev] = 0
        #carry = 1 # just continue to 'bump' the next (reversed) digits
def code2vec(code):
    ret = np.zeros(hid, dtype=np.float32)
    dim = 0
    for i in range(code.shape[0]):
        sel = code[i]
        ret[dim+sel] = 1.0
        dim += nfactors[i]
    return ret

def pool_str(x, d=hid):
    if x.ndim == 1:
        x = x.reshape((1,x.shape[0]))
    s = ""
    for i in range(x.shape[0]):
        dim = 0
        ss = ""
        sep = "" if i==0 else "\n "
        for j in range(len(factors)):
            for k in range(nfactors[j]):
                if x[i,dim+k]:
                    #print(i,j,k,factors[j])
                    ss += sep + factors[j][k]
                    sep = " "
                    break
            #sep = " "
            dim += nfactors[j]
        s = s + ss
    return s
            
a = create_pool(3)
print("a",a.shape,"\n",a)
print("pool_str(a[0])",pool_str(a[0]))
print("pool_str[a], a.shape",a.shape,"\n",pool_str(a))
print("Generating sequential codes")
print("code      vector                                          string")
code = np.array([0,0,0,0], dtype=np.int32)
for i in range(10):
    v = code2vec(code)
    print(code, v, pool_str(v))
    code_up(code)

def create_all():  # since ndistinct is only 96, create one of each "full universe"
    """ return a full universe of items """
    ret = np.zeros((ndistinct,hid), dtype=np.float32)
    code = np.array([0,0,0,0], dtype=np.int32)
    for i in range(ndistinct):
        v = code2vec(code)
        ret[i,:] = v
        code_up(code)
    return ret
univ = create_all()
ustr = [pool_str(univ[i,:]) for i in range(univ.shape[0])]
print("\nfirst and last in universe:")
print(univ[0,:], pool_str(univ[0,:]))   # aka ustr[0]
print(univ[-1,:], pool_str(univ[-1,:])) # aka ustr[-1]

snitch = rng.choice(ndistinct)
goods = np.array([], dtype=np.int32)
print("snitch",snitch,pool_str(univ[snitch]))
#

[0.5, 0.5, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.5, 0.5, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
ret.shape (3, 15) zeros
0 0 0 1
0 1 2 0
0 2 10 1
0 3 12 2
1 0 0 0
1 1 2 7
1 2 10 1
1 3 12 1
2 0 0 0
2 1 2 3
2 2 10 0
2 3 12 0
a (3, 15) 
 [[0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0.]]
pool_str(a[0]) small blue rubber sphere
pool_str[a], a.shape (3, 15) 
 small blue rubber sphere
 large yellow rubber cylinder
 large gray metal cube
Generating sequential codes
code      vector                                          string
[0 0 0 0] [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.] large blue metal cube
[0 0 0 1] [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.] large blue metal cylinder
[0 0 0 2] [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.] large blue metal sphere
[0 0 1 0] [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.] large blue rubber cube
[0 0 1 1] [1. 0

### Universe distances
How would UMAP clustering organize these factors?

In [4]:
%%time
def epoch_normal(pts):
    # torch: pts.sub_(pts.mean(axis=0))
    pts -= np.mean(pts,axis=0)
    d2 = np.average(np.sum(np.square(pts), axis=1))
    print("avg d2", d2)
    pts *= (1.0/np.sqrt(d2))
# note: we might also do an svd and rotate to some "standard" orientation

emb0 = umap.UMAP(
    n_neighbors=15, learning_rate=0.5, random_state=12345, init="random", min_dist=0.001,
    output_constrain={'final_pt': epoch_normal},
).fit_transform(univ)
# alternative, could just do it by hand...
#epoch_normal(emb0)

output_constrain keys dict_keys(['final_pt'])
avg d2 10.760185
CPU times: user 6.55 s, sys: 7.94 ms, total: 6.56 s
Wall time: 6.57 s


In [5]:
def plotit(emb0, snitch, goods):
    #output_file("iris2a.html")

    #targets = [str(d) for d in iris.target_names]
    #targets += ["good","bad"]
    source = ColumnDataSource(
        data = dict(
            x0=emb0[:,0],
            y0=emb0[:,1],
            #label=[ustr[d] for d in range(emb0.shape[0])],
        )
    )
    #for i in range(len(iris.feature_names)
    #    source.data[iris.feature_names[i]] = iris.data[i,]
    # 4 ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
    #source.data["Sepal_Length"] = iris.data[:,0]
    #source.data["Sepal_Width"]  = iris.data[:,1]
    #source.data["Petal_Length"] = iris.data[:,2]
    #source.data["Petal_Width"]  = iris.data[:,3]
    source.data["text"] = [ustr[d] for d in range(emb0.shape[0])]
    tooltips = [
        ("idx", "$index"),
        ("(x,y)",  "(@x0,@y0)"),   # tooltips[1] can be modified in later plots
        ("Text", "@text"),
        #("Sepal Length,Width", "@Sepal_Length{0.0}, @Sepal_Width{0.0}"),
        #("Petal Length,Width", "@Petal_Length{0.0}, @Petal_Width{0.0}"),
    ]
    #print(tooltips[0])

    p1 = figure(title="Test UMAP on Clevr text",
                tooltips=tooltips)
    circles = p1.circle(source=source, x="x0", y="y0",
        size=8, fill_alpha=0.5,
        #color={"field": "label", "transform": cmap},
        #legend_label="species",
        #legend_group="label"
    )
    # tooltips are only for circles
    hover = p1.select_one(HoverTool)
    hover.renderers = [circles]

    # gray boxes around snitch and goods
    cmap = CategoricalColorMapper(factors=["snitch","goods"], palette=Category10[10])
    gb = np.vstack([emb0[snitch,], emb0[goods,]])
    gbcat = np.hstack((np.repeat("snitch",1), np.repeat("good",goods.shape[0])))
    gbsource = ColumnDataSource( dict(
            x0 = gb[:,0],
            y0 = gb[:,1],
            label = gbcat,
        ))
    p1.square(source=gbsource, x="x0", y="y0",
              size=16, line_alpha=0.7, line_width=4, fill_alpha=0.0,
              color={"field": "label", "transform": cmap},
              legend_group="label",
    )
    #p1.add_layout(p1.legend[0], 'right') # outside, plot rectangular!
    #p1.legend.location = 'top_left'
    #p1.legend.location = 'top_right'
    #p1.legend.location = 'center_center'

    return p1

print("nfactors", nfactors, "hid", hid, "--> 2-D,   ndistinct",ndistinct)
fig0 = plotit(emb0, snitch, goods)
show(fig0)

nfactors [2, 8, 2, 3] hid 15 --> 2-D,   ndistinct 96


In [6]:
from scipy.spatial import distance_matrix
import numpy.ma as ma
rr = distance_matrix(emb0,emb0)
print("rr.shape",rr.shape)
def greedy_far_oops(n, emb, perm0=None, verbose=False): 
    assert(n>=1)
    # perm0 represents emb indices to be excluded.
    # These might be previously returned greedy_far values.
    #
    # Internally, pbeg will flag if we are given an initial set
    # of far-away indices, to which we'll add 'n' more.
    
    # We sort and return the n additional selections.
    pbeg = 0 if perm0 is None else perm0.shape[0]
    # To stop if greedy-far permutation is full-length, cap pend
    pend = min(pbeg + n, emb.shape[0])
    perm = np.zeros(pend, dtype=np.int32)
    #lens = np.zeros(n, dtype=np.float32)
    if pbeg:
        perm[0:pbeg] = perm0[0:pbeg]
    
    if pbeg==0:
        # init from scratch: choose first point "closest to origin"
        dists = np.sqrt(np.sum(np.square(emb), axis=1))
        print("dists",dists.shape, dists[0:5])
        i0 = np.argmin(dists)
        print("min dists to origin @", i0, "is", dists[i0])
        perm[pbeg] = i0
        pbeg = pbeg + 1
    print("pbeg",pbeg,"pend",pend)
    dists = distance_matrix(emb, emb)
    print("dists",dists.shape) # e.g. 96x96
    assert( pbeg > 0 )

    for i in range(pbeg,pend):
        # invariants on perm[0:i]
        if verbose:
            print("\ni")
            for j in range(0,i):
                print("perm",j, perm[j], "emb",emb[perm[j]])
        assert( np.all(perm[0:i] >= 0) )
        assert( np.all(perm[0:i] <= emb.shape[0]) )
        assert( perm[0:i].size == np.unique(perm[0:i]).size )
        if True:
            # oops, this returns the furthest from ONE of the selecteds
            #       but we want the furthest from ANY of the selecteds
            # Alg:
            #   find distances of all points to selecteds perm[0:i]
            #      there will be 96 of these
            #   zero out distances of selecteds (do not select those again)
            #   find "non-perm far point" from each exising perm
            #   select furthest of these "non-perm far points"
            dists_from_sels = dists[perm[0:i],:]
            #print("dists_from_sels", dists_from_sels.shape,"\n",dists_from_sels)
            dists_from_sels[:,perm[0:i]] = 0      # not interest in previous fars
            #print("dists_from_sels", dists_from_sels)
            far = np.argmax(dists_from_sels, axis=1)
            if verbose:
                for j in range(len(far)):
                    print("far distance from",perm[j],"to",far[j],"is",
                          dists_from_sels[j,far[j]], "emb",emb[far[j]])
            idx = np.argmax(far)
            if verbose:
                print("selecting furthest far, point",far[idx],"emb",emb[far[idx]])
            perm[i] = far[idx]
            
        
    # Oh, we cannot "just sort", since user wants to distinguish the
    # "new" n greedy-fars from the "old" perm0 ones
    #perm.sort() #np.sort(perm)
    return perm
oops = greedy_far_oops(12,emb0, verbose=False)
print("these are NOT greedy far",oops)


rr.shape (96, 96)
dists (96,) [1.1206018 0.5575227 1.2683991 1.1240767 0.5968729]
min dists to origin @ 70 is 0.5086673
pbeg 1 pend 12
dists (96, 96)
these are NOT greedy far [70  2 69 87 81 63 93 71 65 51 75 59]


In [7]:
def greedy_far(n, emb, perm0=None, verbose=False):
    assert(n>=1)
    # perm0 represents emb indices to be excluded.
    # These might be previously returned greedy_far values.
    #
    # Internally, pbeg will flag if we are given an initial set
    # of far-away indices, to which we'll add 'n' more.
    
    # We sort and return the n additional selections.
    pbeg = 0 if perm0 is None else perm0.shape[0]
    # To stop if greedy-far permutation is full-length, cap pend
    pend = min(pbeg + n, emb.shape[0])
    
    perm = np.zeros(pend, dtype=np.int32)
    #lens = np.zeros(n, dtype=np.float32)
    if pbeg:
        perm[0:pbeg] = perm0[0:pbeg]
    
    if pbeg==0:
        # init from scratch: choose first point "closest to origin"
        dists = np.sqrt(np.sum(np.square(emb), axis=1))
        print("dists",dists.shape, dists[0:5])
        i0 = np.argmin(dists)
        print("min dists to origin @", i0, "is", dists[i0])
        perm[pbeg] = i0
        pbeg = pbeg + 1
    print("pbeg",pbeg,"pend",pend)
    
    # NOTE: for repeated calls to greedy_far,
    #       this distance matrix is CONSTANT (inefficiency here)
    # Slightly better for now:
    #       ask for more "greedy fars" than you absolutely need
    # Best: alt call ignoring emb and using distance matrix instead
    #       (init for that usage might need to change)
    dists = distance_matrix(emb, emb)
    print("dists",dists.shape) # e.g. 96x96
    assert( pbeg > 0 )
    lambdas = np.zeros(pend)    # optional, could be used to threshold "farness"
    idx = perm[0]
    ds = dists[idx,:]           # initialize with a row distances
    #ds = np.minimum(ds, dists[idx,:]) # adjust with col distances
    for i in range(1,pbeg):
        idx = perm[i]
        if verbose: print("init i",i,"idx",idx,"ds[idx]",ds[idx])
        lambdas[i] = ds[idx]
        ds = np.minimum(ds, dists[idx,:]) # <-- pull in these column distances
    for i in range(pbeg,pend):
        idx = np.argmax(ds)
        perm[i] = idx
        if verbose: print("iter i",i,"idx",idx,"ds[idx]",ds[idx])
        lambdas[i] = ds[idx]
        ds = np.minimum(ds, dists[idx,:]) # <-- clever, tricky
    if verbose:
        print("lambdas",lambdas)
        print("perm",perm)
    #return (perm, lambdas)
    return perm
# Often bandits will perform an initial clustering
# and "select one exemplar from each cluster"
#
# Instead of clustering, initial selection can come from "greedy furthest".
# These is quick'n'dirty, even if perhaps non-optimal.
#
# For example, asking for n=12 greedy fars will give one point in each cluster
# for this simple universe, because the initial embedding happened to have
# 12 lo-D clusters.

# twelve is nice to simulate accumulated info straightaway
nchoices = 12

# Since Euclidean hi-D distances are all equivalent, use
# lo-D distances, emb0
choices = greedy_far(nchoices,emb0, verbose=True)

if False:  # double check with some other code
    def getGreedyPerm(D, init=0):
        """
        A Naive O(N^2) algorithm to do furthest points sampling

        Parameters
        ----------
        D : ndarray (N, N) 
            An NxN distance matrix for points
        init : [0] integer in 0..N-1
            Which point to choose first
        Return
        ------
        tuple (list, list) 
            (permutation (N-length array of indices), 
            lambdas (N-length array of insertion radii))
        """

        N = D.shape[0]
        #By default, takes the first point in the list to be the
        #first point in the permutation, but could be random
        perm = np.zeros(N, dtype=np.int64)
        lambdas = np.zeros(N)
        perm[0] = init
        ds = D[init, :]
        for i in range(1, N):
            idx = np.argmax(ds)
            perm[i] = idx
            lambdas[i] = ds[idx]
            ds = np.minimum(ds, D[idx, :])
        return (perm, lambdas)
    (a,l) = getGreedyPerm(distance_matrix(emb0, emb0), init=choices[0])
    print("a",a)
    #print("l",l)
    print("a[0:nchoices]",a[0:nchoices])
    print("  cf. choices",choices)
    print("l[0:nchoices]",l[0:nchoices])
    choices = a[0:nchoices]

if False:
    a = greedy_far(20, emb0, perm0=choices) # test: choose 20 more
    print("20 more", a)
    a = greedy_far(20, emb0, perm0=a)
    print("20 more", a)
    a = greedy_far(20, emb0, perm0=a)
    print("20 more", a)
    a = greedy_far(20, emb0, perm0=a)
    print("20 more", a)
    a = greedy_far(20, emb0, perm0=a)  # check: when run out, we just stop
    print("20 more", a)
    a = greedy_far(20, emb0, perm0=a)  # check: when run out, we just stop
    print("20 more", a)
print("snitch",snitch,pool_str(univ[snitch]))
print("choices",choices,"\n",pool_str(univ[choices]))
#print("lens",lens)

dists (96,) [1.1206018 0.5575227 1.2683991 1.1240767 0.5968729]
min dists to origin @ 70 is 0.5086673
pbeg 1 pend 12
dists (96, 96)
iter i 1 idx 2 ds[idx] 1.7722623712003107
iter i 2 idx 39 ds[idx] 1.4980221122295188
iter i 3 idx 48 ds[idx] 1.4512089900046903
iter i 4 idx 31 ds[idx] 0.7946504384557463
iter i 5 idx 65 ds[idx] 0.7785065535034066
iter i 6 idx 79 ds[idx] 0.7550601750751991
iter i 7 idx 5 ds[idx] 0.7433248368511458
iter i 8 idx 69 ds[idx] 0.7370629353164426
iter i 9 idx 80 ds[idx] 0.7071013422856857
iter i 10 idx 6 ds[idx] 0.698691548734438
iter i 11 idx 46 ds[idx] 0.6525293565444791
lambdas [0.         1.77226237 1.49802211 1.45120899 0.79465044 0.77850655
 0.75506018 0.74332484 0.73706294 0.70710134 0.69869155 0.65252936]
perm [70  2 39 48 31 65 79  5 69 80  6 46]
snitch 41 large red rubber sphere
choices [70  2 39 48 31 65 79  5 69 80  6 46] 
 small gray rubber cylinder
 large blue metal sphere
 large red rubber cube
 small blue metal cube
 large purple metal cylinder
 s

In [8]:
fig1 = plotit(emb0, snitch, choices)
show(fig1)

Note that the hi-D points have many equidistant neighbors.  The lo-D embedding is
very flexible because it cannot possible represent this with fidelity.  *Closeness*
in the above zero-information embedding *has little meaning*.

So we may expect that some *apparently far* choices may end up selected.

In [53]:
# Model a reasonably carefully player. Choose nothing if no factors match,
# or a select all/one with max number of matching factors.  Factors are equally
# important.  Chooser never makes a mistake.
#
# Given a selection of hid-vectors (n * hid)
# select one at random that has a largest number of correct factors
# if no factors are correct, return -1
# else return selection in [0,n-1]
def player_choice(x, cmp = univ[snitch,:], maxsel=999, pmiss=0.0, truthful=None, verbose=True):
    """ return indices in x that best matches snitch (or -1).
    
    Parameters
    ----------
    x:   ndarray(n x hid) options presented to player.
    cmp: the snitch, to witch we compare matching factors
    all: max selections to make
    pmiss: after all, give each selection a chance to be missed
         ? replace by a lower quality match, if there's only one best ?
    truthful: never lie about these selections (set this to the snitch value!)
    """
    assert(len(x.shape) == 2)
    if truthful is None:
        truthful = []
    print("choosing, pmiss",pmiss,"truthful",truthful)
    nmatch = np.sum(np.logical_and(cmp,x), axis=1)
    sels = []
    best = np.amax(nmatch)
    if best==0:
        best = -1
    sel = np.where(nmatch==best)  # where returns a tuple, 1 vector per dim
    sel = sel[0]
    if verbose:
        print("nmatch",nmatch, best, sel)
    if best == -1:
        return sel
    nmatch[sel] = -1   # remove sel from further consideration
    nbest = sel.size
    nerr = 0
    if pmiss > 0.0:
        for i in range(sel.size):
            choice_i = choices[sel[i]]
            if choice_i in truthful:
                #print("being truthful about choice",choice_i)
                pass
            elif rng.random() < pmiss:
                #print("erring about choice",choice_i)
                sel[i] = -1
                nerr += 1
        nmiss = np.sum(sel == -1)
        sel = sel[sel != -1]
        # if empty, can we substitute a single lesser positive nmatch?
        if nmiss == sel.size:
            best = np.amax(nmatch)
            if best==0:
                best = -1
            sel = np.where(nmatch==best)
            sel = sel[0]
            nmatch[sel] = -1
            if sel.size > 1:
                sel = np.random.choice(sel, size=1)
                nerr += 1
    if sel.size > maxsel: # random select...
        sel = np.random.choice(sel, size=maxsel, replace=False)
        nbest = sel.size
        if verbose: print("where-->",sel,"nbest",nbest)
    #if all == True or best == -1:
    #    return sel
    #else:
    #    selend = np.int32(sel.shape[0])
    #    #print("selend",selend)
    #    sel1 = rng.choice(selend)
    #    #print("sel1",sel1)
    #    return sel[sel1]
    if verbose:
        print("selection errors:", nerr)
    return sel

# choose one from all choices:
#(choices,lens) = greedy_far(2,emb0)
options = univ[choices,:]
#print("options\n",options)
#print("snitch\n", univ[snitch,:])

sel = player_choice(options, pmiss=0.0)
print("  SNITCH",snitch,"\n",pool_str(univ[snitch]))
print(" CHOICES",choices,"\n",pool_str(univ[choices]))
print("SELECTED",sel,choices[sel],"\n",pool_str(univ[choices[sel]]))
#print("\n", options[sel,:],"\n",univ[choices[sel]])

if False:
    sel_just1 = player_choice(options, maxsel=1, pmiss=0.5)
    #print("sel_just1",sel_just1, options[sel_just1,:])
    print("sel_just1",sel_just1,choices[sel_just1],pool_str(univ[choices[sel_just1]]))
    
    # choose from 3 with zero common features
    options = univ[[3,5,9],:]  # adjust by hand to test
    #print("options\n",options)
    #print("snitch\n", univ[snitch,:])
    sel_none = player_choice(options, pmiss=0.5)
    #print("sel_none",sel_none, options[sel_none,:])
    print("sel_none",sel_none,choices[sel_none],pool_str(univ[choices[sel_none]]))
#

choosing, pmiss 0.0 truthful []
nmatch [0 4 2 2] 4 [1]
selection errors: 0
  SNITCH 74 
 small green metal sphere
 CHOICES [45 74 78 67] 
 large yellow rubber cube
 small green metal sphere
 small purple metal cube
 small gray metal cylinder
SELECTED [1] [74] 
 small green metal sphere


In [35]:
def embed_higher_dim(x, val=0):
    """ increase the dimension of the last dim of x by one,
        filling it with 'val'.
    """
    return np.concatenate( (x, np.zeros( x.shape[:-1]+(1,))), axis=x.ndim-1 )
embz = embed_higher_dim(emb0, 0.0)  # shape (96,2) --> (96,3)
print(embz.shape, embz[0:3])

adj_up = 0.5
adj_down = -0.01 # maybe even zero is OK
print("adj_up,adj_down",adj_up,adj_down)
def center_last_dim(x):
    """ shift mean of final dim to zero """
    x[...,-1] -= np.mean(x[...,-1])
    return x
def adjust_last_dim(x, choices, sel, up=adj_up, down=adj_down):
    """ Given choices in universe index, and subset sel of choices,
        adjust 'z' of x to promote choices[sel].
        
        Modifies x, and centers average 'z' to ~ 0.
    """
    ndown = len(choices)
    nup = len(sel)
    x[choices,-1] += adj_down
    if len(sel):
        x[choices[sel],-1] += adj_up
    center_last_dim(x)
print(np.mean(embz[:,-1]))
print("choices",choices,"sel",choices[sel])
#print("choices",tuple(choices))
print("emb0",emb0.shape,"\n",emb0[0:4])
print("embz",embz.shape,"\n",embz[0:4])
adjust_last_dim(embz, choices, sel)
print(np.mean(embz[:,-1]))
print("embz",embz.shape,"\n",embz[0:4])

#

(96, 3) [[0.81866193 0.76520652 0.        ]
 [0.08541713 0.55094051 0.        ]
 [0.21047699 1.25081408 0.        ]]
adj_up,adj_down 0.5 -0.01
0.0
choices [17 68 21 44] sel [68]
emb0 (96, 2) 
 [[ 0.8186619   0.7652065 ]
 [ 0.08541713  0.5509405 ]
 [ 0.210477    1.2508141 ]
 [ 1.1237116  -0.0286476 ]]
embz (96, 3) 
 [[ 0.81866193  0.76520652  0.        ]
 [ 0.08541713  0.55094051  0.        ]
 [ 0.21047699  1.25081408  0.        ]
 [ 1.12371159 -0.0286476   0.        ]]
1.1564823173178713e-18
embz (96, 3) 
 [[ 0.81866193  0.76520652 -0.00479167]
 [ 0.08541713  0.55094051 -0.00479167]
 [ 0.21047699  1.25081408 -0.00479167]
 [ 1.12371159 -0.0286476  -0.00479167]]


In [36]:
import pandas as pd
embz = embed_higher_dim(emb0, 0.0)  # shape (96,2) --> (96,3)
adjust_last_dim(embz, choices, sel)

def plotitz(embz, snitch, choices, sel, labels=None, label_names=None, title=None):
    # NOTE:  sel might be passed as "choices[sel]" to be a subset
    #output_file("iris2a.html")

    COLORS = Viridis256          # this is a tuple
    #print("COLORS ", COLORS)
    N_COLORS = len(COLORS)
    #LABELS = Category10          # this is a dict with keys 3..10
    #N_LABELS = max(LABELS.keys())
    LABELS = Viridis
    N_LABELS = 11                 # Viridis has keys 3..11 and 256
    #print("LABELS",LABELS)
    #print(LABELS.keys())
    #print("N_LABELS",N_LABELS)
    #targets = [str(d) for d in iris.target_names]
    #targets += ["good","bad"]
    source = ColumnDataSource(
        data = dict(
            x=embz[:,0],
            y=embz[:,1],
            z=embz[:,2],
            text = [ustr[d] for d in range(embz.shape[0])]
        )
    )
    tooltips = [
        ("idx", "$index"),
        ("(x,y,z)",  "(@x,@y,@z)"),   # tooltips[1] can be modified in later plots
        ("Text", "@text"),
    ]
    if title is None:
        title = "UMAP on Clevr, z ~ color"
        
    pz = figure(title=title, tooltips=tooltips)
    if len(set(source.data['z'])) > N_COLORS:
        # use qcut for quantiles
        groups = pd.cut(source.data['z'], N_COLORS, duplicates='drop')
    else:
        groups = pd.Categorical(source.data['z'])
    c = [COLORS[xx] for xx in groups.codes]
    #print("c",c)
    
    mycmap = linear_cmap(field_name='z', palette='Viridis256',
                         low=np.min(source.data['z']),
                         high=np.max(source.data['z']))

    clabel = 'black'
    lw = 2
    sz = 12
    kwargs={}
    if labels is not None:
        # expect set(labels) = {0,1,2} quantiles or so, select line_color accordingly
        # TODO TODO TODO
        llen = len(set(labels))
        if llen < N_LABELS:
            print("Cat labels, llen",llen)
            groups = pd.Categorical(labels)
        else:
            # use qcut for quantiles
            groups = pd.cut(labels, N_LABELS, duplicates='drop')
        #print("label groups.codes",groups.codes) # 0,1,2,...
        clabel = [LABELS[max(3,llen)][xx] for xx in groups.codes]
        source.data['clabel'] = clabel
        kwargs['line_color'] = 'clabel'
        kwargs['line_width'] = 4
        if label_names is None:
            source.data['lcode'] = groups.codes # integer 0,1,2 in legend
            kwargs['legend_group'] = 'lcode'
        else: # or by text, according to circle line_color "clabel"
            assert( len(label_names) == max(groups.codes)+1 )
            source.data['ltext'] = [label_names[xx] for xx in groups.codes]
            kwargs['legend_group'] = 'ltext'
        
    #print(*kwargs)
    circles = pz.circle(
        source=source, x="x", y="y",
        size=sz, fill_alpha=0.5,
        color = mycmap, fill_color = mycmap,
        
        #line_color = lmap,
        #line_color = clabel,
        #line_color = source.data['ccode']
        #line_color = {'field':'clabel'},
        #line_color = {'field':'ccode'},
        #line_color = {'field':'ccode', 'transform':lmap},
        
        #line_width = lw,
        #color = c, fill_color=c, line_color=None,
        #hover_color="black", hover_alpha=0.5,
        #color = mycmap,
        #color={"field": "z", "transform": COLORS},
        #legend_label="species",
        #legend_group=,
        **kwargs
    )
    # tooltips are only for circles
    hover = pz.select_one(HoverTool)
    hover.renderers = [circles]

    # gray boxes around snitch and goods
    #choices_not_sel = choices[not sel in choices
    choices = np.setdiff1d(choices, sel)  # unique choices that are not in sel
    cmap = CategoricalColorMapper(factors=["snitch","choices","sel"], palette=Category10[10])
    gb = np.vstack([embz[snitch,], embz[choices,], embz[sel]])
    gbcat = np.hstack((np.repeat("snitch",1),
                       np.repeat("choices",choices.shape[0]),
                       np.repeat("sel",sel.shape[0])))
    gbsource = ColumnDataSource( dict(
            x0 = gb[:,0],
            y0 = gb[:,1],
            label = gbcat,
        ))
    pz.square(source=gbsource, x="x0", y="y0",
              size=20, line_alpha=0.7, line_width=8, fill_alpha=0.0,
              color={"field": "label", "transform": cmap}, alpha = 0.5,
              legend_group="label",
    )

    return pz

# set up some example labels (plotting feature test)
exlabs = np.zeros(embz.shape[0], dtype=np.int32)
exlabs[0:int(ndistinct/2)] = 1
exlabs[int(ndistinct*0.5):int(ndistinct*0.75)] = 2
fig2 = plotitz(embz, snitch, choices, choices[sel],
               #labels=exlabs, #label_names=['q0','q1','q2'], # feature test
               title="Clevr UMAP 3-D init state")
show(fig2)
#

In [37]:
# now 3-D constrain the sel points
# This MIGHT be good if we have a bunch of sel points, o.w. NOT GOOD
def show_univ(emb, lo=0, hi=None, univ=univ):
    if hi is None: hi = lo+1        # show single lo item (default if hi is absent)
    elif hi < 0: hi = emb.shape[1]  # show lo..end        (if hi < 0)
    for i in range(lo,hi):
        print("pt",i,'%-50s'%(str(emb[i,:])),pool_str(univ[i]))
print("embz sample (init)")
for i in range(len(sel)):
    show_univ(embz, lo=choices[sel[i]]-2, hi=choices[sel[i]]+3)

# A data constraint limits 'z' range to match the init probabilities of embz,
#   but lets x and y vary freely (we'll constrain individual points during 'fit')
# UMAP-constraints provides a helper that does this.
# Without this constraint, UMAP *might* embed far points at higher-than-selected
# goodness.  We want the [history of] selected points to be at highest goodness.
def mk_bound_last_dim(embz):
    """ Return a jitted fn(vector1d[ndim]) that bounds the final dimension
        (ndim-1) of vector1d to the min--max values of array embz[...,ndim].
    """
    bound_los = np.empty((embz.shape[-1]), dtype=np.float32)
    bound_his = np.empty((embz.shape[-1]), dtype=np.float32)
    # bound none of the first dimensions ...
    bound_los[0:-1] = +1.0
    bound_his[0:-1] = -1.0
    # set last dimension bound ...
    bound_los[-1] = np.min(embz[...,-1]) # lo
    bound_his[-1] = np.max(embz[...,-1]) # hi
    print("bound_los",bound_los)
    print("bound_his",bound_his)
    # create a numba function, referencing local parameters
    @numba.njit()
    def fn_last_dim_range(pt):
        return umap.constraints.dimlohi_pt(pt, bound_los, bound_his)
    # this function does NOT depend on 'idx' arg
    return fn_last_dim_range

embedder3 = umap.UMAP( n_components=3,
    n_neighbors=50, learning_rate=0.1, random_state=12346, init=embz,
    negative_sample_rate=5, repulsion_strength=0.40,
    min_dist=0.001, spread=3.0,
    output_constrain = {'pt': mk_bound_last_dim(embz)},
    #a=0.1, b=0.9,
)
# A simple constraint
pin_sel = np.ones(embz.shape[0])
pin_sel[choices[sel]] = 0.0
print("pin_sel",pin_sel)
emb3 = embedder3.fit_transform(embz, data_constrain=pin_sel)

print("emb3 (sel constrained)")
for i in range(len(sel)):
    show_univ(emb3, lo=choices[sel[i]]-2, hi=choices[sel[i]]+3)
fig3 = plotitz(emb3, snitch, choices, choices[sel],
               title="first-step dist-based coloring + pinned %d selected pts"%(len(sel)))
show(fig3)
print("emb3 choices...")
for i in range(len(choices)):
    show_univ(emb3,lo=choices[i])

embz sample (init)
pt 66 [-1.14358699  0.43228638 -0.00479167]              small gray metal cube
pt 67 [-0.69736826  0.20842332 -0.00479167]              small gray metal cylinder
pt 68 [-1.22213411 -0.00196824  0.48520833]              small gray metal sphere
pt 69 [-0.13929413 -1.22894692 -0.00479167]              small gray rubber cube
pt 70 [-0.16456863 -0.48131025 -0.00479167]              small gray rubber cylinder
bound_los [ 1.          1.         -0.01479167]
bound_his [-1.         -1.          0.48520833]
pin_sel [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
X.shape (96, 3)
init.shape           (96, 3)
data_constrain.shape (96,)
output_constrain keys dict_keys(['pt'])
pin_mask 96 (96,)
pin_mask 1d: 1 points don't move
emb3 (sel cons

emb3 choices...
pt 17 [ 9.181104   -4.233252    0.44000468]              large cyan rubber sphere
pt 68 [-1.2221341  -0.00196824  0.48520833]              small gray metal sphere
pt 21 [ 9.558392   -1.3750232   0.07891221]              large gray rubber cube
pt 44 [ 2.924676e+00  8.763969e+00 -6.525299e-03]        large yellow metal sphere


## above CAN be really bad
- especially with <= 1 pinned point
- even with 2 pins, **sometimes** has given huge probability to seemingly distant points.
  For example, yellows and purples might still be fairly mixed.  
  A partial fix was done by constraining the 'z' "probability" range to min/max of selected points.    
  But it **still** can show high-P points scattered quite more likely than should

- We really want a high-probability cluster to form
  - Do this with *supervised UMAP*, since the selection "distances" seem actually
    to have a nice plot.
  - Begin be defining a *similarity-to-selected" distance
    - Easily defined by an aggregate of (Euclidean 3-D) distances to some set
      high-weight selections
    - Because UMAP *does* reflect to high dimensional distance as Euclidean in lo-D


In [38]:
#
# What if we used ONLY user-dim values to select points?
#   i) we might not have enough selecteds
#      (ex. distribution has 1 high selected, rest ~ 0)
#   ii) non-selecteds (low prob) have essentially zero information
#       about "distance" to the selecteds
# But let's try it out anyway
#
def rescale_prob(x, T=None):
    """ return probability distribution for x converted to 0,1 range """
    # TODO: add a temperature-like rescaling to choose between exploit/explore
    lo = np.min(x)
    hi = np.max(x)
    p = (x - lo)  * (1.0/(hi-lo))
    p *= (1.0 / np.sum(p))
    return p

# choose according to current "probability distribution" (z)
p = rescale_prob(emb3[:,2].copy())
print("sum p",np.sum(p))
# Since "game" is continuing, previous selection was not snitch.
# remove immeditately previous selection (it definitely was not the snitch)
# (could remove more if we know user has not changed his snitch definition)
print("snitch:",pool_str(univ[snitch]))
print("old choices",choices)
print("old sel within choices",sel)
print("old sel within universe",choices[sel])
print("old sel",pool_str(univ[choices[sel]]))
print("p(old sel)    %",np.int32(p[choices[sel]]*1000.0)*0.1)
print("removing old sel probability[ies]")
p[choices[sel]] = 0.0
p = rescale_prob(p)
print("sum p",np.sum(p))
print("probabilities %",np.int32(p*1000.0)*0.1)
print("p(old sel)    %",np.int32(p[choices[sel]]*1000.0)*0.1)
choices_usel = np.random.choice(np.arange(emb3.shape[0]), size=4, replace=False, p=p)
print("choices_usel",choices_usel)
print(pool_str(univ[choices_usel]))
sel_usel = player_choice(options, pmiss=0.0)
fig3_usel = plotitz(emb3, snitch, choices_usel, choices[sel],
                    title="new choice & selection via user-dim value only")
show(fig3_usel)
#
# Of course we end up selecting some "yellows" that *look* far away.
# Even we did now pick out the snitch, the visualization looks bad.
#
# naah.  What I want is a threshold classifier, and choose
# some from "snitch-like" (exploit) class, and rest from "not-snitch" (explore)
# say npos exploit and nneg explore
# Threshold set as quantiles 'q' of 'p' values
# Ex. q = [0.25,0.5,0.75] quartiles to direct UMAP clustering a bit better?
#
# Then embz should use the q-class as a clustering aid.
#

sum p 1.0
snitch: small green metal sphere
old choices [17 68 21 44]
old sel within choices [1]
old sel within universe [68]
old sel small gray metal sphere
p(old sel)    % [2.1]
removing old sel probability[ies]
sum p 1.0
probabilities % [1.9 2.  1.7 2.1 0.1 2.1 0.  0.1 2.1 0.  2.  0.2 1.2 1.7 0.  1.9 0.1 1.9
 0.5 2.1 0.  0.4 0.  1.9 0.  0.  0.5 2.1 0.  0.6 2.1 0.  2.1 0.  1.9 0.
 0.2 0.  0.8 0.  2.  0.  2.1 2.  0.  1.7 2.1 1.7 2.  2.  0.1 0.  0.1 1.7
 0.  1.7 2.1 1.9 0.1 2.1 0.  2.1 2.1 2.1 2.  1.1 1.9 0.6 0.  0.  2.  0.
 0.6 0.  0.2 0.1 2.  0.  2.1 0.6 2.  0.  0.  2.  0.  0.1 0.  0.4 2.1 0.1
 0.5 2.1 2.1 2.1 0.4 0.2]
p(old sel)    % [0.]
choices_usel [ 0 76 47 91]
large blue metal cube
 small green rubber cylinder
 large yellow rubber sphere
 small yellow metal cylinder
choosing, pmiss 0.0 truthful []
nmatch [1 3 0 2] 3 [1]
selection errors: 0


# Redo emb0 processing with new strategy
We want a nice visualization that:

- ranks all points more smoothly wrt. similarity to selecteds
- visually clusters selecteds wrt. this similarity
- still represents hi-D topology

A simple possibility is to use the "too bimodal" user dims
with a very high-temperature selection.  However this still fails
in the extreme case of only two distinct user-dim values.  This
extreme case throws away *all* the topological information!

Fortunately, the 3-D Euclidean UMAP embedding already provides us
with a lo-D distance measure that reflects both selecteds and topology. So we returns to emb0/embz/emb3 path, but with:

- a *distance-to-selecteds* distance similarity measure that
  is smoother than just the user-dim values.
- whose distances can be coarsely *quantized*
- to generate a few *class labels*
- that loosely guide a *supervised UMAP* embedding to respect both hi-D topology and the similarity-to-selecteds measurements

TODO: we really want a way to access the UMAP topological loss value.  When topological loss deteriorates it is a strong signal that the "raw" hi-D representation (with its default metric) is failing to capture the feature\[s\] the user is really interested in.  Then we need to escalate out analysis: (i) hi-D weighted Cartesion metric vs other metric choices? (ii) Some neural net remappings of hi-D features? (iii) expand hi-D features from raw data s.t. mutual info is added and new hi-D topology helps explain the user clustering?

In [39]:
import sklearn

emb4 = embed_higher_dim(emb0, 0.0)  # shape (96,2) --> (96,3)
adjust_last_dim(emb4, choices, sel)

# First time -- there are no significant quantiles, but we
# do have an approximate Euclidean "distance to closest selected"
# that distinguishes "equivalent" unselected points.
def min_dist_to(x, chosen):
    """ for each x[i], calculate min(dist(x[i],x[chosen]))
        for a list of chosen points.
        
        Parameters
        ----------
        x: array nsamples x ndim
        chosen: array of sample indices
        
        Return
        ------
        sim : array nsamples
              similarity measure based on distance to set of chosen
    """
    return np.min(sklearn.metrics.pairwise.euclidean_distances(x, x[chosen,:]),
                  axis = 1)
# When min_dist_to give 0.0, this is highest probability, flip and rescale:
def similarity_to(x, chosen, alg='min'):
    """ For every sample in x[nsamples x ndim], find its similarity
        to a subset x[chosen,:].  Returned similarity values for chosen
        indices should be high.
        
        Similarity values are rescaled to fill range [0,1] when possible.
        
        Parameters
        ----------
        x : array nsamples x ndim
        chosen : array of sample indices
        alg : 'min'|'sum'|'avg'
              How to agglomerate multiple distances to items in a set
              
        Return
        ------
        array[nsamples] of similarities-to-set
        
        Notes:
        min-similarity is the usual "distance from a point to a set".
        
        Sum/avg-similarity really is an experiment.  I wanted things 'inside'
        to have high weight, but I'm not sure this does this.  Points in the
        subset can end up having differing and non-maximal distance-to-set.
        Crazy.
    """
    sim = None
    if len(chosen):
        if alg == 'min':
            dalg = min_dist_to(x, chosen)    # vector of x.shape[0]
            lo = np.min(dalg)
            hi = np.max(dalg)
            if hi > lo:
                sim = 1.0 - (dalg-lo)*(1.0/(hi-lo))  # sim in [0,1]
        elif alg == 'sum' or alg == 'avg':
            # EXPERIMENTAL: probably not a good idea
            dists = sklearn.metrics.pairwise.euclidean_distances(x, x[chosen,:])
            #print("dists",dists.shape,"\n",dists)
            dalg = np.sum(dists, axis = 1)
            #print("dalg.shape",dalg.shape,dalg)
            lo = np.min(dalg)
            hi = np.max(dalg)
            if hi > lo:
                sim = 1.0 - (dalg-lo)*(1.0/(hi-lo))  # sim in [0,1]
        else:
            raise ValueError('similarity_to alg %s not supported [min|sum|avg]', alg)
    if sim is None:
        sim = 0.5 * np.ones(x.shape[0])
    return sim
    
pt1 = np.random.uniform(size=(3,2))
pt2 = np.array([[0,0],[1,1]], dtype=np.float32)
print("pt1", pt1.shape, pt1)
print("pt2", pt2.shape, pt2)
print("dists to [0,0]",
      sklearn.metrics.pairwise.euclidean_distances(pt1,Y=np.array([[0.0,0.0]])))
dists = sklearn.metrics.pairwise.euclidean_distances(pt1,Y=pt2)
print("dists pt1 to set pt2",dists)
print("min dist axis 1",np.min(dists, axis=1))

pt3 = np.random.uniform(size=(9,2))
subset = [0,1,2]
pt3[len(subset)] = np.mean(pt3[subset])
print("pt3\n",pt3)
print("min similarity of pt3 to subset",subset,"\n",
      similarity_to(pt3, subset, alg='min'))
print("sum similarity of pt3 to subset",subset,"(a bit crazy)\n",
      similarity_to(pt3, subset, alg='sum'))

# trial 1
#embzsim = similarity_to(emb0, choices[sel])
#print(embzsim[choices[sel]]) # 1.0 (or maybe 0.5)
#zlo = np.min(embz[:,2])
#zhi = np.max(embz[:,2])
#assert(zhi > zlo)
#emb4[:,2] = np.sqrt(np.square((embz[:,2] - zlo) * (1.0/(zhi-zlo))) *
#                    np.square(embzsim))
# trial 2
emb4sim = similarity_to(emb4, choices[sel], alg='min')
emb4a = emb4.copy()    #temporarily
emb4a[:,2] = emb4sim

# pre-labelling plot
fig4a = plotitz(emb4a, snitch, choices, choices[sel],
                title="emb4a sumdist-coloring (this is a temporary construct)")
show(fig4a)

pt1 (3, 2) [[0.66774282 0.65904469]
 [0.11504175 0.98638137]
 [0.34419779 0.05421803]]
pt2 (2, 2) [[0. 0.]
 [1. 1.]]
dists to [0,0] [[0.9382006 ]
 [0.99306738]
 [0.34844183]]
dists pt1 to set pt2 [[0.9382006  0.47607285]
 [0.99306738 0.88506303]
 [0.34844183 1.15090403]]
min dist axis 1 [0.47607285 0.88506303 0.34844183]
pt3
 [[0.39444463 0.53735292]
 [0.40563381 0.93647254]
 [0.22959964 0.32599418]
 [0.47158295 0.47158295]
 [0.04472921 0.74315609]
 [0.68130792 0.18098258]
 [0.66955718 0.57250518]
 [0.62527127 0.67475543]
 [0.59609224 0.06041866]]
min similarity of pt3 to subset [0, 1, 2] 
 [1.         1.         1.         0.77841639 0.11301982 0.
 0.39374916 0.4128152  0.01067184]
sum similarity of pt3 to subset [0, 1, 2] (a bit crazy)
 [1.         0.69380915 0.80320681 0.84494935 0.49636283 0.10905033
 0.52990688 0.60786175 0.        ]


## This similarity is smoother, and...
we see a nice yellowish gradient away from selections (and snitch)

can define a nice probability for future selections (at some Temperature)  
can be used to cluster disparate selections together  
  - Note that the topological embedding has many degrees of freedom,
    and can give several different lo-D embeddings of similar quality.  
  - We ask UMAP to promote a particular lo-D embedding that **also** creates
    a visual *cluster*

We chose a similarity measure that points *between* selected items in the
initial embedding higher weight than those *outside* our current selections.  
Note that we still need to add parameters to reasonably shrink weights on
*historical* selections (we have no history yet in this demo)

## Sometimes our selecteds still ended up "too separated"
So we define some loose quantiles based on our similarity measure  
(maybe related to exploit/explore status and to how many choices we wish to present?).
Esp. focus on the upper quantiles  
(TODO: weight the class labellings to do this)

In [40]:
# Let's create some labels according to quantiles of emb4[:,2] (z)
print(emb4a.shape)
qu = np.quantile(emb4a[:,2],q=[0.0,0.25,0.5,0.75,0.9,1.0])
print(qu.shape, qu)
thresh = qu[-2]
print(thresh)
labels = np.int32(emb4a[:,2] > qu[-2])
print(labels[choices[sel]]) #should be 1
print(labels)
#for (i,lab) in enumerate(labels):
#    if labels[i]:
qu90 = np.argwhere( emb4a[:,2] > qu[-2] )[:,0]
print("snitch",pool_str(univ[snitch]))
print("sel",choices[sel],"\n",pool_str(univ[choices[sel]]))
print("qu90",qu90,"\n",pool_str(univ[qu90]))



(96, 3)
(6,) [0.         0.18796456 0.35775206 0.59393053 0.71649756 1.        ]
0.7164975630431436
[1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0]
snitch small green metal sphere
sel [68] 
 small gray metal sphere
qu90 [50 56 62 66 68 74 80 86 90 92] 
 small blue metal sphere
 small brown metal sphere
 small cyan metal sphere
 small gray metal cube
 small gray metal sphere
 small green metal sphere
 small purple metal sphere
 small red metal sphere
 small yellow metal cube
 small yellow metal sphere


### and repeat, doing a **supervised umap**
with these 5 quantiles (labels) as rough topological clustering guides

In [41]:
# repeat, emb3 code, but as supervised labeling
# let's use all quantiles to redirect the UMAP clustering
labels = np.zeros(emb4a.shape[0], dtype=np.int32)
for i in range(1, qu.size):
    labels += (emb4a[:,2] > qu[i])
print(labels[choices[sel]]) #should be "high"
print(labels)
print(np.argwhere(labels == 4)[:,0])

# now 3-D constrain the sel points
# UMAP suggests n_neighbors=15 for supervised embedding
# We could also label with -1 to denote 'class unknown'
embedder5 = umap.UMAP( n_components=3,
    n_neighbors=15, learning_rate=0.1, random_state=12346, init=emb4a,
    negative_sample_rate=5, repulsion_strength=0.40,
    min_dist=0.001, spread=3.0,
    #output_constrain = # may want a 3-D box [-10,+10] constraint on all 3 lo dims?
    #a=0.1, b=0.9,
)
# do not PIN selected, but assign them with supervised clustering
#pin_sel = np.ones(embz.shape[0])
#pin_sel[choices[sel]] = 0.0
#print("pin_sel",pin_sel)
#emb3 = embedder3.fit_transform(embz, data_constrain=pin_sel)
emb5 = embedder5.fit_transform(embz, y=labels)
print(emb5[0:4])
fig5 = plotitz(emb5, snitch, choices, choices[sel], labels=labels,
               title="supervised 90%-ile, original choices and selection")
show(fig5)

[4]
[0 2 1 0 1 0 0 2 1 0 1 0 0 1 1 0 1 0 0 2 1 0 1 0 0 2 1 0 1 0 0 1 1 0 1 0 0
 1 1 0 1 0 0 1 1 0 1 0 3 3 4 1 2 2 3 3 4 2 2 2 3 3 4 1 2 2 4 3 4 1 2 2 3 3
 4 2 2 2 3 3 4 1 2 2 3 3 4 2 2 2 4 3 4 2 2 2]
[50 56 62 66 68 74 80 86 90 92]
[[11.213413  12.535436  -1.6149366]
 [ 6.2777753  8.011782   5.470405 ]
 [ 4.0978923 14.453775   1.3469133]
 [14.039931   4.214662  -3.1380107]]
Cat labels, llen 5


### Now the top 10% form a nice cluster for "exploit"
and unselected choices have mostly moved quite far away.

Unfortunately I have to mouse-hover to see the yellow top decile cluster as
really having something like about 10% of the universe of size 96 :(

In [42]:
# Demo some ways to set up next choices
if False:
    # in this case both "converge" to the snitch, but maybe one is better?
    
    # alt method: just use the (x,y) of the top cluster...
    # this throws away some fine-grained cluster distance info.
    print("xy proj", emb5[qu90,0:2])
    qmid = np.mean(emb5[qu90,0:2], axis=0)
    print("qmid",qmid)
    # top cluster, less prev non-winning selections
    qu90less = np.setdiff1d(qu90, choices[sel])
    print("qu90less",qu90less)
    for i in range(qu90less.size):
        #print(qu90less[i], emb5[qu90less[i]], qmid)
        print(qu90less[i], np.sqrt(np.sum(np.square(emb5[qu90less[i],0:2]-qmid))))
    dmid = np.sqrt(np.sum(np.square(emb5[qu90less,0:2]-qmid), axis=1))
    print(dmid, np.argmin(dmid))
    choice_mid = qu90less[np.argmin(dmid)]
    print("choice_mid", choice_mid, pool_str(univ[choice_mid]))

    # find closest to cluster center, excluding previous selections
    print(emb5[qu90,:])
    qmid = np.mean(emb5[qu90,:],axis=0)     # center of full top quantile
    print("qmid",qmid)
    # top cluster, less prev non-winning selections
    qu90less = np.setdiff1d(qu90, choices[sel])
    print("qu90less",qu90less)
    for i in range(qu90less.size):
        #print(qu90less[i], emb5[qu90less[i]], qmid)
        print(qu90less[i], np.sqrt(np.sum(np.square(emb5[qu90less[i],:]-qmid))))
    dmid = np.sqrt(np.sum(np.square(emb5[qu90less,:]-qmid), axis=1))
    print(dmid, np.argmin(dmid))
    choice_mid = qu90less[np.argmin(dmid)]
    print("choice_mid", choice_mid, pool_str(univ[choice_mid]))

def choice_mid(emb, clust, sel, n=1):
    """ For exploit, choose up to n [1] index of clust (not including sel) whose
        emb[index] is closest to the full-cluster full-dim mean.
        
        Increase 'n' when favoring exploit.
        For explore, "rest" of items via Temperature-related probability ('z', or better
        a smoother similarity score based on 3-D Euclidean distance to {weighted selecteds}).
        
        I.e. index of a central xyz-exemplar of emb[clust], not including sel
        TBD: n>1
        TBD: list of exclusions 'sel' should accumulate and need soft/hard reset!
             From first selection forward, sel increases in size as long
             as user selects > 1 item.  Until we find the snitch or do a hard reset
             forgetting all previous sel.
             After a snitch event, or a sudden return to zero selections, we reset,
             because maybe the user changed his mind or forgot his snitch.
    """
    middle = np.mean(emb[clust,:], axis=0)
    #print("mid",middle)
    subset = np.setdiff1d(clust, sel)   # exclude 'sel' from cluster
    if subset.size == 0:  # reasonable edge case behavior
        subset = clust
    #print("subset",subset)
    dists2 = np.sum(np.square(emb[subset,:]-middle), axis=1)
    central_idx = np.argmin(dists2)
    if n > 1:
        # TBD: <=n lowest indices
        #args = np.argsort(dists2)
        #for i in range(len(args)):
        #    print("ith", i, "dist^2", dists2[args[i]], subset[args[i]])
        # or even better, try argpartition(dists2, kth=n)
        #kth = int(subset.shape[0]/2)
        kth = n
        idxs = np.argpartition(dists2, kth=kth)
        #for i in range(len(idxs)):
        #    print("ith", i, "dist^2", dists2[idxs[i]], subset[idxs[i]])
        # We can post-sort the selected (initial) partition
        part = subset[idxs[:kth]]
        pdst = dists2[idxs[:kth]]
        print(dists2[idxs[:kth]])
        resort = np.argsort(pdst)
        central_idxs = part[resort]
    else:
        central_idxs = np.array([central_idx], np.int32)
    
    #print("central_idx",central_idxs)
    return subset[central_idx]

q90mid = choice_mid( emb5, qu90, choices[sel])
print("prev choices",choices,"\n",pool_str(univ[choices]))
print("prev selection", choices[sel],"\n",pool_str(univ[choices[sel]]))
print("snitch",snitch,pool_str(univ[snitch]))
print("AFTER ONE SELECTION:")
print("central in non-selected 90%-ile", pool_str(univ[q90mid]))


# at high 'exploit', central might present several choices.
# SHOULD select by dists2 rank

prev choices [17 68 21 44] 
 large cyan rubber sphere
 small gray metal sphere
 large gray rubber cube
 large yellow metal sphere
prev selection [68] 
 small gray metal sphere
snitch 74 small green metal sphere
AFTER ONE SELECTION:
central in non-selected 90%-ile small green metal sphere


In [43]:
#
# Now set the prev steps as a game:
#
# Init: nchoices, rand init guess,
#       max_select 'one' or 'all'
#       1st round special-cased (to get finer-grained distance)
#
# Iteration:
#   - state: emb_prev, choices_prev, sel_prev
#   - if sel_prev is snitch:  END
#   - exploit: depends on len(sel_prev) (zero means exploit lower)
#   - emb_now: supervised UMAP embedding based on quantiles
#   - choices_now = [central of highest cluster less sel_prev]
#   - choices_now += select from distribution of quantiles, using exploit factor
#   - choices_now --> sel_now(max_select)
#   - (emb_prev, choices_prev, sel_prev) <-- (emb_now, choices_now, sel_now)
#
# If no selection, go toward to random quantile selection, assuming user may have
# decided on a new snitch.
#
# Actually could use bandit ideas, but here only "in spirit"
#
# Simulation parameters
sim_n = 4          # choices presented
sim_selmax = 1     # max selections
sim_selmiss = 0.0  # chance per selection to mistakenly miss it
sim_round = 0      # count choice presentations
# universe and utility functions as above.
sim_adj_up = 0.5
sim_adj_down = -0.01 # maybe even zero is OK
# TODO: When xy dims of UMAP embeddings have a "scale", sim_adj 
#       directly relates to the importance of user selections (vs. topology)
#       emb2d does use an epoch_normal standardization, but not sure if this
#       will happen in later stages.
print("sim adj_up,adj_down",sim_adj_up,sim_adj_down)
def sim_adjust_last_dim(x, downweight, upweight, up=sim_adj_up, down=sim_adj_down):
    """ Given choices in universe index, and subset sel of choices,
        adjust 'z' of x to promote choices[sel].
        
        Parameters
        ----------
        x : 2d array nsamples x ndim
            we modify values in last of ndim
        downweight: list of samples to downweight
        upweight:   list of samples to upweight
        
        Returns
        -------
        None. Modifies x, and centers average 'z' to ~ 0.
    """
    ndown = len(downweight)
    nup = len(upweight)
    if ndown:
        x[downweight,-1] += down
    if nup:
        x[upweight,-1] += up
        print("up",upweight,"weights",x[upweight,-1])
    center_last_dim(x)


print("    initialize a",univ.shape[1],"--> 2D UMAP embedding")
emb2d = umap.UMAP(
    n_neighbors=15, learning_rate=0.5, random_state=12345, init="random", min_dist=0.001,
    output_constrain={'final_pt': epoch_normal},
).fit_transform(univ)


sim adj_up,adj_down 0.5 -0.01
    initialize a 15 --> 2D UMAP embedding
output_constrain keys dict_keys(['final_pt'])
avg d2 10.760185


In [60]:
print("INIT PHASE (special, random far until we get some info)")
rng = default_rng(seed=1234569)
snitch = rng.choice(ndistinct)
# Until we get SOME INFO, present random choices
# The random choices probably should not repeat
fars = np.array([], dtype=np.int32)
while True:
    print("snitch",snitch,pool_str(univ[snitch]))
    sim_round += 1
    print("    ROUND",sim_round)
    #choices = greedy_far(sim_n,emb2d,choices)
    nprev = fars.size
    fars = greedy_far(sim_n,emb2d,fars)
    choices = fars[nprev:]
    if choices.size == 0: # restart random choices
        print("restarting random picks")
        fars = np.array([], dtype=np.int32)
        fars = greedy_far(sim_n,emb2d,fars)
        choices = fars
    # FIXME: REMOVE previous rand choices from consideration,
    #        i.e. use smaller and smaller subset of emb2d
    #print("choices",choices,"\n",pool_str(univ[choices]))
    sel = player_choice(univ[choices,:], cmp=univ[snitch,:],
                        maxsel=sim_selmax,
                        pmiss=sim_selmiss,
                        #pmiss=0.9,            # set this high to demo random rounds
                        truthful=[snitch],
                        verbose=True)
    for i in range(len(choices)):
        print(choices[i], "sel" if i in sel else "   ", ustr[choices[i]])
    # Note: at high pmiss, we might present the snitch and user makes a mistake!
    if sel.size:
        print("    SELECTED", pool_str(univ[choices[sel]]))
        print("    (snitch)", pool_str(univ[snitch]))
        break
    print("    SELECTED none")

    
#print("round",sim_round,"selected",sel,pool_str(univ[choices[sel]]))
print("WIN" if snitch in choices[sel] else "(snitch not yet found)")
emb3d = embed_higher_dim(emb2d, 0.0)  # shape (96,2) --> (96,3)

if True:
    fig0 = plotit(emb2d, snitch, fars)
    show(fig0)
    fig1 = plotitz(emb3d, snitch, fars, choices[sel], title="UMAP init 3d embedding")
    show(fig1)


INIT PHASE (special, random far until we get some info)
snitch 74 small green metal sphere
    ROUND 107
dists (96,) [1.1206018 0.5575227 1.2683991 1.1240767 0.5968729]
min dists to origin @ 70 is 0.5086673
pbeg 1 pend 4
dists (96, 96)
choosing, pmiss 0.0 truthful [74]
nmatch [1 2 0 2] 2 [1 3]
where--> [3] nbest 1
selection errors: 0
70     small gray rubber cylinder
2     large blue metal sphere
39     large red rubber cube
48 sel small blue metal cube
    SELECTED small blue metal cube
    (snitch) small green metal sphere
(snitch not yet found)


In [61]:
print("Found a first selection")
# Instead use the Euclidean 3D distance to the full set of user-selected points
# With a smoothish similarity-to-set measure to define a next embedding.
#
# Quantiles of these guide a SUPERVISED umap, modifying the previous init
# (hopefully lightly, so n_epochs=1 might be possible)

# Here we might downweight the full set 'fars', or the final set 'choices'
print("snitch",snitch)
print("fars",fars)
print("choices",choices)
selected = choices[sel]
print("selected", selected)
def did_we_win(selected):
    return np.any(selected == snitch)
print("WIN" if snitch in selected else "(snitch not yet found)")
emb3d = embed_higher_dim(emb2d, 0.0)  # shape (96,2) --> (96,3)
# mul by 0.5 because selected go up twice, and we want to keep relative scale "decent"
sim_adjust_last_dim(emb3d, fars, selected, up=sim_adj_up*0.5, down=sim_adj_down*0.5)
sim_adjust_last_dim(emb3d, choices, selected, up=sim_adj_up*0.5, down=sim_adj_down*0.5)

fig3da = plotitz(emb3d, snitch, fars, selected,
                 title="UMAP init 3d, color ~ adjusted user dim")
show(fig3da)


# create similarity-to-selected
embsim = similarity_to(emb3d, selected)

print("selected",selected)
print("emb3d[selected]\n",emb3d[selected,:])
print("embsim[selected] (ones?)\n", embsim[selected])
print("embsim",embsim)
emb4a = emb3d.copy()    #temporarily
print("emb3d[selected]",emb3d[selected,:])
print("emb4a[selected]",emb4a[selected,:])
emb4a[:,-1] = embsim
fig4a = plotitz(emb4a, snitch, choices, selected,
                title="UMAP init 3d, color ~ mindist")
show(fig4a)

Found a first selection
snitch 74
fars [70  2 39 48]
choices [70  2 39 48]
selected [48]
(snitch not yet found)
up [48] weights [0.245]
up [48] weights [0.48760417]


selected [48]
emb3d[selected]
 [[-1.16198778  0.57280672  0.48520833]]
embsim[selected] (ones?)
 [1.]
embsim [0.21474521 0.48642178 0.38296171 0.07513891 0.25108662 0.03701867
 0.20272251 0.4637186  0.4224938  0.11734936 0.22487188 0.05063516
 0.27004658 0.44349623 0.38029636 0.05601029 0.20978153 0.
 0.21006268 0.43730289 0.41697125 0.02777559 0.25653125 0.07796825
 0.25337966 0.4600461  0.4392006  0.09980503 0.2648939  0.08834564
 0.23203899 0.42138338 0.38233135 0.06824659 0.28538182 0.07873044
 0.25256975 0.42931793 0.34595845 0.04089161 0.24778519 0.0504033
 0.27002211 0.39710254 0.39608746 0.09480744 0.27435654 0.03227387
 1.         0.66704493 0.69436796 0.22926973 0.38803472 0.40763612
 0.80379174 0.67018684 0.68140974 0.25487544 0.3624463  0.36747931
 0.80919831 0.67388931 0.7059384  0.21564893 0.35949655 0.38315506
 0.80455615 0.70600128 0.70968538 0.18427935 0.41187507 0.34906934
 0.80829232 0.6966587  0.73532451 0.27172013 0.39127983 0.39862521
 0.79409663 0.64885855 0.6703

In [62]:
def emb_labels(embsim):
    """ Generate some similarity labels.
    
    Parameters
    ----------
    embsim: vector[nsamples] of similarity values
    Return
    ------
    label vector[nsamples] integers from 0 to 4
    """
    qu = np.quantile(embsim,q=[0.0,0.25,0.5,0.75,0.9,1.0])
    labels = np.zeros(embsim.shape, dtype=np.int32)
    for i in range(1, qu.size):
        labels += (embsim > qu[i])
    #print("embsim",embsim)
    #print("qu",qu)
    #print("qu labels",labels)
    return labels

# re-embed using mindist similarity to cluster
def emb_mindist_clustering(emb, choices, selected, method=0, plot=False):
    """ Return a next 3D embedding using user-dim info emb[:,-1].
    
    Parameters
    ----------
    emb : array[nsamples x 3] with last dim = user selection weights
    choices : sample numbers of original choices
    selected : sample numbers of selections
    method : algorithm variants default=0 expends the least effort
        Methods 0,1,2 embed first into 2D and then add back
                      the user dimension of emb unchanged.
                      This info may be rather coarse until many +ve selections accrue
        Methods 3,4,5 embed directly into 3D, changing the user dimension info at will
        Methods 0,3 cluster primarily with quantization labels
        Methods 1,4 use hi-D original features + raw user dim
        Methods 2,5 use hi-D original features + a smoother similarity feature
    """
    # update emb by a smooth similarity
    embsim = similarity_to(emb, selected)
    # We cluster embsim, since it is "smoother"
    # (or we could do temperature-based reweighting of emb[:-1])?
    # (also, higher labels should be "more important")
    labels = emb_labels(embsim)
    
    emb_clust = None
    if method==0 or method==1 or method==2:
        # methods with 2D umap output
        init = emb[:,0:2]
        embed_clust = umap.UMAP(
            n_components=2, init=init,
            n_neighbors=15, learning_rate=0.1, random_state=12346,
        )
        
        if method==0:
            # 3D to 2D embedding with ONLY label data
            emb_clust2 = embed_clust.fit_transform(emb, y=labels)
        elif method==1 or method==2:
            # expanded univ to 2D embedding
            univx = np.empty((univ.shape[0], univ.shape[1]+1))
            univx[:,0:univ.shape[1]] = univ
            if method==1:
                univx[:,-1] = emb[:,-1]
            else:
                univx[:,-1] = embsim
            emb_clust2 = embed_clust.fit_transform(univx, y=labels)
        # add back user dimension of original
        emb_clust = embed_higher_dim(emb_clust2,0.0)
        emb_clust[:,-1] = emb[:,-1]
        
    elif method==3 or method==4 or method==5:
        init = emb.copy()
        init[:,-1] = embsim #optional
        # actually, it is more important to cluster highest label value
        # as this represents the points closest to selected.

        embed_clust = umap.UMAP(
            n_components=3, init=init,
            n_neighbors=15, learning_rate=0.1, random_state=12346,
            # optional:
            #   Since init[:,-1] is bounded to [0,1], we can constrain the fit
            #   However, for method 4 this can yield infinities
            output_constrain = {'pt': mk_bound_last_dim(init)},
        )
        if method==3:
            # Do we rely just on the quantization labels?
            emb_clust = embed_clust.fit_transform(univ, y=labels)
        elif method==4 or method==5:
            # add emb[-1] user data as a new hi-dim feature
            univx = np.empty((univ.shape[0], univ.shape[1]+1))
            univx[:,0:univ.shape[1]] = univ
            if method==4:
                univx[:,-1] = emb[:,-1]
            else:
                univx[:,-1] = embsim
            emb_clust = embed_clust.fit_transform(univx, y=labels)
    assert(emb_clust is not None)
            
    # do not PIN selected, but assign them with supervised clustering
    # Well, perhaps constrain up to to "selected" to be xy-pinned
    #pin_sel = np.ones(embz.shape[0])
    #pin_sel[choices[sel]] = 0.0
    #emb2d = umap.UMAP(
    #    n_neighbors=15, learning_rate=0.5, random_state=12345, init="random", min_dist=0.001,
    #    output_constrain={'final_pt': epoch_normal},
    #).fit_transform(univ)
    
    #print("emb_clust[0:4]",emb_clust[0:4])
    #print("emb_clust[snitch]",emb_clust[snitch])
    #print("emb_clust[selected]",emb_clust[selected])
    fig_clust = plotitz(emb_clust, snitch, choices, selected, labels=labels,
                        title="supervised quantiles, method %d" % (method))
    if plot:
        show(fig_clust)
    #print("top cluster",emb_clust[labels==4])
    #print("next cluster",emb_clust[labels==3])
    return (emb_clust,labels,fig_clust)

# For transition to game loop
if snitch in selected:
    print("OHOH, let us put the already-presented snitch back into play")
    fars = fars[fars!=snitch]
    choices = choices[choices!=snitch]
    selected = selected[selected!=snitch]
print("snitch",snitch)
print("fars",fars)
print("choices",choices)
print("selected", selected)
    

emb3d = embed_higher_dim(emb2d, 0.0)  # shape (96,2) --> (96,3)
# mul by 0.5 because selected go up twice, and we want to keep relative scale "decent"
sim_adjust_last_dim(emb3d, fars, selected, up=sim_adj_up*0.5, down=sim_adj_down*0.5)
sim_adjust_last_dim(emb3d, choices, selected, up=sim_adj_up*0.5, down=sim_adj_down*0.5)

prev_choices = np.zeros(emb3d.shape[0], dtype=bool)
prev_selected = np.zeros(emb3d.shape[0], dtype=bool)

# Normally if the user got a snitch, he must select it
# But if he missed it, we should not exclude previous "fars"
#prev_choices[fars] = True         # initial rounds where user selected nothing

# GAME LOOP: This concentrates on exploitation
def game_loop(sim_round,emb3d, choices, selected, prev_choices, prev_selected):
    prev_choices[choices] = True      # 1st round where user selected something
    prev_selected[selected] = True    # actually a subset of choices, so no effect
    # Q: get rid of fars in lieu of prev_choices ?
    #last_choices = choices.copy()
    #last_selected = selected.copy()

    # default method gives emb_next with unchanged user-info, but clustered xy
    (emb_next, labels, fig) = emb_mindist_clustering(emb3d, choices, selected)
    # next set of choices could use: euclidean emb_next distance-to-set
    # Some temperature-related selection based on "raw" coarse user-dim info
    # or ...
    #    ... For Euclidean min-dist, re-use similarity_to ...
    sim_round += 1
    print("    GAME LOOP ROUND",sim_round)
    print("np.where prev_selected",np.where(prev_selected)[0])
    
    # This does not forget previous (poorer) guesses anywhere fast enough
    #psim = similarity_to(emb_next, np.where(prev_selected)[0])
    psim = similarity_to(emb_next, selected)
    
    psim[prev_selected] = 0.0 # weak, but ok if user had failed to correctly select the snitch!
    #psim[prev_choices] = 0.0  # even stronger whittling down of choice pool.
    
    assert( np.sum(psim>0.0) > sim_n ) # did we run out of presentation choices
    #prob = psim/np.sum(psim)
    #print("prob",prob)
    #print("sum prob", np.sum(prob))
    choices = rng.choice(emb_next.shape[0], size=sim_n, replace=False, p=psim/np.sum(psim))
    #print("choices",choices,"\n",pool_str(univ[choices]))
    sel = player_choice(univ[choices,:], cmp=univ[snitch,:],
                            maxsel=sim_selmax,
                            pmiss=sim_selmiss,
                            #pmiss=0.9,            # set high for testing inattentive users
                            truthful=[snitch],
                            verbose=True)

    selected = choices[sel]
    for i in range(len(choices)):
        print(choices[i], "sel" if i in sel else "   ", ustr[choices[i]])
    #if sel.size:
    #    print("    SELECTED", pool_str(univ[selected]))
    #    print("    (snitch)", pool_str(univ[snitch]))
    #else:
    #    print("    SELECTED none")
    print("snitch", pool_str(univ[snitch]))
    
    # update user dim weights HERE
    sim_adjust_last_dim(emb_next, choices, selected)
    
    print("WIN" if snitch in selected else "(snitch not yet found)")
    return (sim_round, snitch in selected, emb_next, choices, selected,
            prev_choices, prev_selected, labels, fig)

emb = emb3d.copy()
for i in range(30):
    (sim_round,win,emb, choices,selected, prev_choices,prev_selected,
     labels,fig) = game_loop(sim_round, emb, choices,selected, prev_choices, prev_selected)
    if win:
        break
        




snitch 74
fars [70  2 39 48]
choices [70  2 39 48]
selected [48]
up [48] weights [0.245]
up [48] weights [0.48760417]
Cat labels, llen 5
    GAME LOOP ROUND 108
np.where prev_selected [48]
choosing, pmiss 0.0 truthful [74]
nmatch [3 2 2 1] 3 [0]
selection errors: 0
86 sel small red metal sphere
54     small brown metal cube
59     small brown rubber sphere
31     large purple metal cylinder
snitch small green metal sphere
up [86] weights [0.48520833]
(snitch not yet found)
Cat labels, llen 5
    GAME LOOP ROUND 109
np.where prev_selected [48 86]
choosing, pmiss 0.0 truthful [74]
nmatch [2 4 1 0] 4 [1]
selection errors: 0
44     large yellow metal sphere
74 sel small green metal sphere
58     small brown rubber cylinder
34     large purple rubber cylinder
snitch small green metal sphere
up [74] weights [0.48041667]
WIN


### All emb_mindist_clustering methods are ~identical, so far

In [ ]:
m0 = emb_mindist_clustering(emb3d, choices, selected, method=0, plot=False)
m1 = emb_mindist_clustering(emb3d, choices, selected, method=1, plot=False)
m2 = emb_mindist_clustering(emb3d, choices, selected, method=2, plot=False)
m3 = emb_mindist_clustering(emb3d, choices, selected, method=3, plot=False)
m4 = emb_mindist_clustering(emb3d, choices, selected, method=4, plot=False)
m5 = emb_mindist_clustering(emb3d, choices, selected, method=5, plot=False)

from bokeh.layouts import gridplot
grid = gridplot(children = [m0[2], m3[2], m1[2], m4[2], m2[2], m5[2]],
                ncols=2,
                #width=250,
                #height=250,
               )
show(grid)

### Haifeng:
After many users, we might be able to generate an initial clustering
based on some set of cross-user disparate clusters.  This is then a
great "initial embedding" that quickly concentrates the user into any
one of the "predetermined" natural clusters.  **user intent**

In [ ]:
a = np.random.rand(2,2,2,2)
a = np.random.rand(3,3)
print(a)
print("a.min(),max()",np.min(a),np.max(a))
print(a[...,-1])
print("a[...,-1] min,max",np.min(a[...,-1]),np.max(a[...,-1]))
